Imports

In [2]:
from plantfusion.l_egume_wrapper import L_egume_wrapper # L-egume FSPM management
from plantfusion.lgrass_wrapper import Simpraise_wrapper # L-grass FSPM management : TO BE IMPLEMENTED
# see : https://github.com/mwoussen/plantfusion/blob/develop/simulations/other_fspm.py
from plantfusion.light_wrapper import Light_wrapper # light management using LightVegeManager
from plantfusion.soil_wrapper import Soil_wrapper # soil management
from plantfusion.indexer import Indexer # index management
from plantfusion.planter import Planter # plant position management

import time
import datetime
import os

Simulation legume alone (default)

In [ ]:
in_folder = "inputs_soil_legume"
out_folder = "outputs/legume_default"
id_usm = 1711

plants_name = "legume"
indexer = Indexer(global_order=[plants_name], legume_names=[plants_name])
planter = Planter(generation_type="default", indexer=indexer, legume_cote={plants_name : 40.}, legume_number_of_plants={plants_name : 64})
legume = L_egume_wrapper(name=plants_name, indexer=indexer, in_folder=in_folder, out_folder=out_folder, IDusm=id_usm, planter=planter)
lighting = Light_wrapper(lightmodel="riri5", indexer=indexer, planter=planter, legume_wrapper=legume)
soil = Soil_wrapper(out_folder=out_folder, legume_wrapper=legume, legume_pattern=True)

try:
    for t in range(legume.lsystem.derivationLength):
        legume.derive(t)

        scene_legume = legume.light_inputs(elements="voxels")
        lighting.run(scenes=[scene_legume], energy=legume.energy(), day=legume.doy(), parunit="RG")
        legume.light_results(legume.energy(), lighting)

        (
                N_content_roots_per_plant,
                roots_length_per_plant_per_soil_layer,
                plants_soil_parameters,
                plants_light_interception,
            ) = legume.soil_inputs()
        soil.run(
            legume.doy(),
            [N_content_roots_per_plant],
            [roots_length_per_plant_per_soil_layer],
            [plants_soil_parameters],
            [plants_light_interception],
        )
        legume.soil_results(soil.results, planter)

        legume.run()

finally:
    execution_time = int(time.time() - current_time_of_the_system)
    print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

    legume.end()


Path management

In [3]:
in_folder_legume = "inputs_soil_legume"
in_folder_lgrass = "inputs_lgrass"
out_folder = "outputs/lgrass_legume"

id1 = 17111
id2 = 17112
writegeo = True

try:
    # Create target Directory
    os.mkdir(os.path.normpath(out_folder))
    print("Directory ", os.path.normpath(out_folder), " Created ")
except FileExistsError:
    print("Directory ", os.path.normpath(out_folder), " already exists")


Directory  outputs\lgrass_legume  Created 


Already implemented in plantfusion (should run if given appropriate parameters)

In [ ]:

legume_name = "legume"
lgrass_name = "lgrass"

indexer = Indexer(global_order=[legume_name, lgrass_name], legume_names=[legume_name], other_names=[lgrass_name])

generation_type = "random"
plant_density = {legume_name : 250, lgrass_name : 450} # plantes.m-2
planter = Planter(generation_type=generation_type, indexer=indexer, plant_density=plant_density)
#note : arg "inter_rows" is not needed for random generation (I think), but is set to 0.15 anyway.

legume = L_egume_wrapper(
    name=legume_name,
    indexer=indexer,
    in_folder=in_folder_legume, out_folder=out_folder,
    planter=planter,
    IDusm=1711,
    caribu_scene=True
)

sky = "turtle46"
lighting_caribu = Light_wrapper(
        lightmodel="caribu",
        indexer=indexer, 
        planter=planter, 
        legume_wrapper=legume,
        sky=sky,
        writegeo=False,
    )


soil = Soil_wrapper(
    in_folder=in_folder_legume, 
    out_folder=out_folder, 
    IDusm=1711, # unit of simulation
    planter=planter, 
    opt_residu=0, 
    save_results=True
)


Simulation legume alone (caribu)

In [ ]:
try:
    current_time_of_the_system = time.time()
    for t in range(legume.lsystem.derivationLength):
        legume.derive(t)

        ### CARIBU
        scene_legume = legume.light_inputs(elements="triangles")
        start = time.time()
        lighting_caribu.run(
            scenes=[scene_legume], day=legume.doy(), parunit="RG"
        )
        caribu_time = time.time() - start
        legume.light_results(legume.energy(), lighting_caribu)

        (
            N_content_roots_per_plant,
            roots_length_per_plant_per_soil_layer,
            plants_soil_parameters,
            plants_light_interception,
        ) = legume.soil_inputs()
        soil.run(
            legume.doy(),
            [N_content_roots_per_plant],
            [roots_length_per_plant_per_soil_layer],
            [plants_soil_parameters],
            [plants_light_interception],
        )
        legume.soil_results(soil.results, planter)

        legume.run()

        print("Lighting running time | CARIBU: ", caribu_time)

    execution_time = int(time.time() - current_time_of_the_system)
    print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

finally:
    legume.end()

Lgrass Wrapper instanciation

In [ ]:

lgrass = Simpraise_wrapper(
    name=lgrass_name,
    indexer=indexer,
    in_folder=in_folder_lgrass, out_folder=out_folder,
    planter=planter,
    plan_sim_file='plan_simulation.csv',
    genet_file='ped.r',
    param_plant_file='liste_plantes.csv'
    #ADD ARGS SPECIFIC TO L-GRASS
)

Simulation Lgrass alone

In [1]:
try:
    current_time_of_the_system = time.time()
    for t in range(legume.lsystem.derivationLength):
        lgrass.derive(t)

        ### CARIBU
        scene_grass = lgrass.light_inputs(elements="triangles")
        start = time.time()
        lighting_caribu.run(
            scenes=[scene_lgrass], day=lgrass.doy(), parunit="RG"
        )
        caribu_time = time.time() - start
        lgrass.light_results(legume.energy(), lighting_caribu)

        (
            N_content_roots_per_plant,
            roots_length_per_plant_per_soil_layer,
            plants_soil_parameters,
            plants_light_interception,
        ) = lgrass.soil_inputs()
        soil.run(
            lgrass.doy(),
            [N_content_roots_per_plant],
            [roots_length_per_plant_per_soil_layer],
            [plants_soil_parameters],
            [plants_light_interception],
        )
        lgrass.soil_results(soil.results, planter)

        lgrass.run()

        print("Lighting running time | CARIBU: ", caribu_time)

    execution_time = int(time.time() - current_time_of_the_system)
    print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

finally:
    legulgrassme.end()

SyntaxError: invalid syntax (3175211231.py, line 3)

In [ ]:


current_time_of_the_system = time.time()

for t in range(legume.lsystem.derivationLength) :# chose derivation lengh

    legume.derive(t)

    # while lgrass.lsystem.current_day > t:
    print(lgrass.lstring)
    lgrass.derive(t)

    scene_legume = legume.light_inputs(elements="triangles")
    scene_lgrass = lgrass.light_inputs() # ARGS DEPENDS ON LGRASS CLASS IMPLEMENTATION
    scenes = indexer.light_scenes_mgmt({lgrass_name : scene_lgrass, legume_name : scene_legume})

    lighting.run(scenes=scenes, day=legume.doy(), parunit="RG") #what is parunit="RG" ?
    legume.light_results(legume.energy(), lighting)
    lgrass.light_results() # ARGS DEPENDS ON LGRASS CLASS IMPLEMENTATION

    soil_legume_inputs = legume.soil_inputs()
    soil_lgrass_inputs = lgrass.soil_inputs()

    (
        N_content_roots_per_plant,
        roots_length_per_plant_per_soil_layer,
        plants_soil_parameters,
        plants_light_interception, # what are these output's type ?
    ) = indexer.soil_inputs({legume_name : soil_legume_inputs, lgrass_name : soil_lgrass_inputs})

    soil.run( # update soil inputs
        legume.doy(), #doy : day of the year (julian day)
        N_content_roots_per_plant,
        roots_length_per_plant_per_soil_layer,
        plants_soil_parameters,
        plants_light_interception,
    )

    #compute new plant parameters for next derivation. To be implemented in Lgrass wrapper
    legume.run()
    lgrass.run()
execution_time = int(time.time() - current_time_of_the_system)
print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

# end() effect is specific to the model
legume.end()
lgrass.end()
soil.end()